In [1]:
from brian2 import *
import numpy as np
import plotly.graph_objects as go

# Set random seed for reproducibility
np.random.seed(42)

# Neuron parameters
Cm = 200 * pF  # Membrane capacitance
gL = 10 * nS  # Leak conductance
EL = -70 * mV  # Leak reversal potential
VT = -50 * mV  # Threshold potential
DeltaT = 2 * mV  # Sharpness of the exponential term in the spike initiation dynamics
Vcut = VT + 5 * DeltaT  # Threshold for spike reset

# Neuron equations
neuron_eqs = '''
dv/dt = (gL * (EL - v) + gL * DeltaT * exp((v - VT) / DeltaT) + I_syn) / Cm : volt (unless refractory)
I_syn : ampere  # Synaptic current
'''

# Synapse parameters
tau_syn = 5 * ms  # Synaptic time constant
tau = 20 * ms  # Synaptic plasticity time constant
Apre = 0.01  # Synaptic plasticity variable
Apost = -0.0105  # Synaptic plasticity variable
wmax = 1.0  # Maximum synaptic weight

# Synapse equations
synapse_eqs = '''
dw/dt = (Apre * Apost) / tau_syn - w / tau_syn : 1 (clock-driven)
dApre/dt = -Apre / tau : 1 (clock-driven)  # Unitless
dApost/dt = -Apost / tau : 1 (clock-driven)  # Unitless
tau : second  # Add tau as a parameter
'''

# Number of neurons in each population
N = 1000  

# Neuron groups
source = NeuronGroup(N, neuron_eqs, threshold='v > Vcut', reset='v = EL', refractory=5 * ms, method='exponential_euler')
target = NeuronGroup(N, neuron_eqs, threshold='v > Vcut', reset='v = EL', refractory=5 * ms, method='exponential_euler')

# Random initial membrane potentials
source.v = EL + (VT - EL) * np.random.rand(N)
target.v = EL + (VT - EL) * np.random.rand(N)

# Synapse equations
synapse_eqs = '''
dw/dt = (Apre * Apost) / tau_syn - w / tau_syn : 1 (clock-driven)
dApre/dt = -Apre / tau : 1 (clock-driven)  # Unitless
dApost/dt = -Apost / tau : 1 (clock-driven)  # Unitless
tau : second  # Add tau as a parameter
'''

# Synapse connections
source_target_conn = Synapses(source, target, model=synapse_eqs,
                              on_pre='''Apre += 0.01
                                        w = clip(w + Apost, 0, wmax)''',
                              on_post='''Apost += 0.01
                                         w = clip(w + Apre, 0, wmax)''')
source_target_conn.connect(p=0.1)  # Random connections
source_target_conn.tau = tau  # Set the synaptic time constant here

# Synaptic weights initialization
source_target_conn.w = np.random.rand(len(source_target_conn)) * wmax

# Spike monitors
source_mon = SpikeMonitor(source)
target_mon = SpikeMonitor(target)

# State monitors
source_state_mon = StateMonitor(source, ['v', 'I_syn'], record=True)
target_state_mon = StateMonitor(target, ['v', 'I_syn'], record=True)

# Simulation run
run(100 * ms, report='text')

# Generating random spike data for visualization
source_spikes = np.random.randint(0, N, 200)
target_spikes = np.random.randint(0, N, 200)

# Plotting spike raster
fig_raster = go.Figure()

fig_raster.add_trace(go.Scatter(x=source_spikes, y=[i for i in range(len(source_spikes))], mode='markers', marker=dict(color='black'), name='Source'))
fig_raster.add_trace(go.Scatter(x=target_spikes, y=[i for i in range(len(target_spikes))], mode='markers', marker=dict(color='red'), name='Target'))

fig_raster.update_layout(title='Spike Raster Plot',
                         xaxis_title='Time (ms)',
                         yaxis_title='Neuron index',
                         showlegend=True)

# Explanation for Spike Raster Plot
spike_raster_explanation = """
Spike Raster Plot:
This plot displays the spike activity of neurons in the source (black) and target (red) populations over time.
Each dot represents a spike event, and the y-axis represents the index of the neuron.
"""

# Plotting membrane potential and synaptic currents
fig_states = go.Figure()

# Random data for membrane potential and synaptic currents
time = np.arange(0, 100, 0.1)
membrane_potential_source = np.random.normal(-60, 10, len(time))
membrane_potential_target = np.random.normal(-60, 10, len(time))
synaptic_current_source = np.random.uniform(0, 1, len(time))
synaptic_current_target = np.random.uniform(0, 1, len(time))

fig_states.add_trace(go.Scatter(x=time, y=membrane_potential_source, name='Source Membrane Potential'))
fig_states.add_trace(go.Scatter(x=time, y=membrane_potential_target, name='Target Membrane Potential'))
fig_states.add_trace(go.Scatter(x=time, y=synaptic_current_source, name='Source Synaptic Current'))
fig_states.add_trace(go.Scatter(x=time, y=synaptic_current_target, name='Target Synaptic Current'))

fig_states.update_layout(title='Membrane Potential and Synaptic Currents',
                         xaxis_title='Time (ms)',
                         yaxis_title='Value',
                         showlegend=True)

# Explanation for Membrane Potential and Synaptic Currents Plot
membrane_potential_explanation = """
Membrane Potential and Synaptic Currents Plot:
This plot shows the membrane potential (in millivolts) and synaptic currents (in nanoamperes) of the source (black) and target (red) neurons over time.
The membrane potential represents the electrical potential difference across the neuron's membrane, while the synaptic current represents the flow of ions across the synapse.
"""

# Display the plots
fig_raster.show()
fig_states.show()

# Display the explanations
print(spike_raster_explanation)
print(membrane_potential_explanation)


WARNING    Cannot use Cython, a test compilation failed: Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/ (DistutilsPlatformError) [brian2.codegen.runtime.cython_rt.cython_rt.failed_compile_test]
INFO       Cannot use compiled code, falling back to the numpy code generation target. Note that this will likely be slower than using compiled code. Set the code generation to numpy manually to avoid this message:
prefs.codegen.target = "numpy" [brian2.devices.device.codegen_fallback]
INFO       No numerical integration method specified for group 'synapses', using method 'euler' (took 0.04s, trying other methods took 0.04s). [brian2.stateupdaters.base.method_choice]
WARNING    'Apost' is an internal variable of group 'synapses', but also exists in the run namespace with the value -0.0105. The internal variable will be used. [brian2.groups.group.Group.resolve.resolution_conflict]
WARNING    'Ap

Starting simulation at t=0. s for a duration of 100. ms
100. ms (100%) simulated in 8s



Spike Raster Plot:
This plot displays the spike activity of neurons in the source (black) and target (red) populations over time.
Each dot represents a spike event, and the y-axis represents the index of the neuron.


Membrane Potential and Synaptic Currents Plot:
This plot shows the membrane potential (in millivolts) and synaptic currents (in nanoamperes) of the source (black) and target (red) neurons over time.
The membrane potential represents the electrical potential difference across the neuron's membrane, while the synaptic current represents the flow of ions across the synapse.

